In [1]:
import numpy as np
from scipy.integrate import quad

def ndquad(func, limits, args=()):
    # Base case: when no more integration dimensions are left, evaluate func.
    if not limits:
        return func(*args)
    else:
        # Take the first integration limit
        lower, upper = limits[0]
        # Integrate over the first variable using quad, and recurse on the remaining dimensions.
        result, err = quad(lambda x: ndquad(func, limits[1:], args + (x,)), lower, upper)
        return result

# Define the integrand for part (b)
def integrand(x, y):
    return np.log(x**2 + y**2 + 1) * np.exp(- (x**2 + y**2))

# Set the integration limits for x and y: from 0 to infinity.
limits = [(0, np.inf), (0, np.inf)]

# Compute the integral using the multidimensional integration function
result = ndquad(integrand, limits)
print("Approximation for the integral:", result)


Approximation for the integral: 0.46837012301477754


In [3]:
import numpy as np
from scipy.integrate import quad
from scipy.special import erfinv
import time

# Recursive n-dimensional quadrature integration using 1D quad
def ndquad(func, limits, args=()):
    if not limits:
        return func(*args)
    else:
        lower = limits[0]
        upper = limits[0]
        result, err = quad(lambda x: ndquad(func, limits[1:], args + (x,)), lower, upper)
        return result

def integrand(x, y):
    return np.log(1 + x**2 + y**2) * np.exp(-(x**2 + y**2))


limits = [(0, np.inf), (0, np.inf)]
start_quad = time.time()
quad_result = ndquad(integrand, limits)
end_quad = time.time()
quad_time = end_quad - start_quad

# =====================
# Monte Carlo Integration via Importance Sampling
# =====================
# We want to sample (x,y) from g(x,y) = (4/pi)*exp(-(x^2+y^2)) for x,y >= 0.
# The marginal density for x is f(x)= (2/sqrt(pi))*exp(-x^2) with CDF F(x)=erf(x).
# So we sample: x = erfinv(u), u ~ Uniform(0,1). (Similarly for y.)
n = 10**5
# Generate uniform random numbers for each coordinate
u = np.random.rand(n)
v = np.random.rand(n)
x_samples = erfinv(u)
y_samples = erfinv(v)

# Compute the importance-sampling ratio:
# f(x,y)/g(x,y) = ln(1+x^2+y^2)*e^{-(x^2+y^2)} / ((4/pi)*e^{-(x^2+y^2)}) = (pi/4)*ln(1+x^2+y^2)
mc_values = (np.pi/4) * np.log(1 + x_samples**2 + y_samples**2)

start_mc = time.time()
mc_estimate = np.mean(mc_values)
end_mc = time.time()
mc_time = end_mc - start_mc

# =====================
# Report results
# =====================
print("Quadrature result: {:.8f}  (computed in {:.4f} seconds)".format(quad_result, quad_time))
print("Monte Carlo result: {:.8f}  (computed in {:.4f} seconds)".format(mc_estimate, mc_time))


Quadrature result: 0.46837012  (computed in 0.0471 seconds)
Monte Carlo result: 0.46885094  (computed in 0.0002 seconds)
